# Lab 5 - Tacoma Businesses

Welcome! At this point, you've seen a number of ways of automating the process of data acquisition, analysis, and visualization. You've also worked your way through the basics of object-oriented programming using python.

You know how to, for example, use pandas to load up a csv, the Arc API (or geopy) to geocode addresses found in that CSV, and then Folium to quickly generate an interactive web map of that data. You also know how to use iteration to parse through large, repetitive data sets to pull out the information you need (remember how many 'uncles' there were?).

There's still ***a lot*** more to learn and programmatic spatial data analysis and visualization is a ***constant*** process of learning new libraries while simultaneously deepening your understanding of others. It can feel daunting, but it's also a lot of fun.

### Enough pre-amble!

The point is, *you've learned quite a lot*. This lab is a chance to show off the skills you, lock down those that make the most sense ***in your workflow***, and learn new ones along the way.

As such, I'm going to give you a problem, an outcome, and tell you where to find some data. The rest is up to you. We'll have two 'themes' and a series of questions that relate to each theme. At the end, you'll produce a visualization. These problems are *contrived*, but designed to force you to acquire, process, and visualize data in ways not unlike those that you'll likely face moving forward.

## That dang taxi lobby!

For this problem, you'll need the **Tacoma Business License** data that you can find at [Results253](https://data.cityoftacoma.org/) and the Washington State House of Representative districts for 2020 available via [the U.S. Census](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html). A couple of notes: 1. The State level House of Representatives is the **lower** chamber and WA's number code is 53 (how would you find this out yourself?) 2. The Tacoma data is **much** easier (in my opinion) to access via the open-data API that it uses.

### Problem 1 - Where are the taxis? (2pts)

Generate an interactive map that displays where all the **taxi services** are registered around Tacoma.

I am going to show you how to dump all of the Business License data into a geodataframe using the open-data API. This technique (of iterating over results) is ***extremely*** useful. Pay attention to what I'm doing, think of it as a gift from your wise and generous professor. Or, just make sure you read it, ok?


In [ ]:
import geopandas 
x = 0
while x <33000:
    url = 'https://data.cityoftacoma.org/resource/29fz-3eiz.geojson?$limit=1000&$offset='+str(x)
    if x == 0:
        df = geopandas.read_file(url)
    else:    
        df2 = geopandas.read_file(url)
        df = df.append(df2)
    x = x + 1000

taxi=df[df['business_name']=='TNC DRIVER']

import folium
map1=folium.Map(location=[47.2029, -122.4443],
                zoom_start=11)
folium.GeoJson(taxi, 
               name="geojson",
              ).add_to(map1)
folium.LayerControl().add_to(map1)

map1

### Problem 2 - Who represents the people!? (2 pts)

Generate an map (interactive or static) that displays **State** House of Representative districts. Hover over should dispaly the district name. Give each district a different color.

In [ ]:
import json, geopandas, zipfile

# extract zipfile
with zipfile.ZipFile('tl_2020_53_sldl20.zip', 'r') as zip_ref:
    zip_ref.extractall('Wa')
shp = "Wa\\tl_2020_53_sldl20.shp"
districts_shp = geopandas.read_file(shp)

#convert shp to geojson
districts_shp.to_file('wa.geojson', driver = 'GeoJSON')
districts_json = json.load(open('wa.geojson', 'r'))

import plotly.express as pe
map2 = pe.choropleth_mapbox( districts_shp,
                             geojson=districts_json,
                             locations="NAMELSAD20",
                             color="SLDLST20",
                             featureidkey='properties.NAMELSAD20',
                             hover_name="NAMELSAD20",
                             mapbox_style="carto-positron",
                             zoom=5.6,
                             center={'lat':47.23, 'lon':-122.2})

map2.update_layout(title_text='Washington by District',
                   showlegend=False)
map2.show()

### Problem 3 - Taxis per square meter of land by district ... wut? (4pts)

Right, so this is a bit contrived, but I want an interactive choropleth map of the number of registered taxi businesses per square meter of land by state House of Representative districts.

You can think of it as finding a good spot to found your own taxi business (based on House districts for some reason) or that you want to know where it would be cheapest to lobby a state rep about your existing taxi business. Or you can think about how it's forcing you to work through a number of interesting spatial analysis to generate interactive visualizations... *whatever seems most fun*.

So, you'll need to do a few things here:
1. You'll need to do some sort of summary statistic to determine *how many* (count) fall into each district.
2. You'll need to create a rate (not count) of licenses per square meter. The ALAND column is square meters of land.

I really like the spatial join functions of geopandas, but *as always* you should work in the workflow that makes sense to you.

**Bonus 1: Have the hover over display the names of the representatives for each district. You can find a list [here](https://apps.leg.wa.gov/Rosters/Members/House). +1 pts** *(The easiest means here is likely a join on the district number, but how will you handle two reps for each district? A concatenated column might work!)*

**Bonus 2: Normalize it based on population. +3 pts** *(You can use 2010 maps if you need to, I won't give many more hints than that)*

In [ ]:
import geopandas 
#load taxi
x = 0
while x <33000:
    url = 'https://data.cityoftacoma.org/resource/29fz-3eiz.geojson?$limit=1000&$offset='+str(x)
    if x == 0:
        df = geopandas.read_file(url)
    else:    
        df2 = geopandas.read_file(url)
        df = df.append(df2)
    x = x + 1000
taxi=df[df['business_name']=='TNC DRIVER']

#load districts
shp = "Wa\\tl_2020_53_sldl20.shp"
districts_shp = geopandas.read_file(shp)

#spatial Joins district to taxi
districts = districts_shp.to_crs("EPSG:4326")
taxi = taxi.to_crs("EPSG:4326")
districts_frame = geopandas.GeoDataFrame(districts)
taxi_frame = geopandas.GeoDataFrame(taxi)
taxi_districts = geopandas.sjoin(taxi_frame, districts_frame, how="inner", op='intersects')

#count taxi in districts
count_taxi_districts = taxi_districts.groupby('NAMELSAD20').size().reset_index(name='counts')

#attribute Joins districts -ALAND column
count_taxi_districts = districts_shp.merge(count_taxi_districts, on='NAMELSAD20')

#count taxi per square meter of land ‰
count_taxi_districts['taxi_per_land(‰)'] = count_taxi_districts.counts/count_taxi_districts.ALAND20*1000

#choropleth map
import plotly.express as pe
districts_json = geopandas.read_file('wa.geojson')
map3 = pe.choropleth_mapbox( count_taxi_districts,
                             geojson=districts_json,
                             locations="NAMELSAD20",
                             color="taxi_per_land(‰)",
                             featureidkey='properties.NAMELSAD20',
                             hover_name="NAMELSAD20",
                             mapbox_style="carto-positron",
                             zoom=5.6,
                             center={'lat':47.23, 'lon':-122.2})
map3.update_layout(title_text='Taxi Per Square Meter of Land by District',)
map3

## Check in (5pts)
### Full credit if you've completed all three problems, partial from there. 

### Problem 4: Fire Insurance 1 - Fires per business (3pts)

Ok, another somewhat contrived example. We'll be working with the Tacoma fire data (from last lab) and the Tacoma business license data (from the other half of this lab). We're going to do some *very poor* 'risk' analysis to determine what businesses are 'most' at risk of a fire.

On the one hand, this sort of analysis - done much more rigorously - forms the backbone of spatial data analysis and its value to corporations and researchers. On the other hand, if you showed *this specific analysis* to someone who models risk, they would have a good chuckle. In other words, we're working through powerful techniques and concepts, but in a kind of 'canned' and silly manner. Enjoy.

Display an interactive, choropleth map based on Neighborhood Council districts that gives the rate of **fires per business license**. In other words, if the East End had 5 fires and 10 licenses, it would get the value 0.5; if Proctor had 20 fires and 5 licenses, it would be 4.... etc. You know how to do this.

**Bonus: Access both data sets using the open data API +1 pts (this actually makes some mapping tasks easier), +1 pt




In [ ]:
import json, pandas, geopandas
#import fire
fire = pandas.read_csv('Fires.csv')

#count fire in neighborhoodCouncil
count_fire_neighborhoodCouncil = fire.groupby('NeighborhoodCouncil').size().reset_index(name='fire_counts')

#import company
x = 0
while x <33000:
    url = 'https://data.cityoftacoma.org/resource/29fz-3eiz.geojson?$limit=1000&$offset='+str(x)
    if x == 0:
        df = geopandas.read_file(url)
    else:    
        df2 = geopandas.read_file(url)
        df = df.append(df2)
    x = x + 1000
company=df[df['city']=='TACOMA']

#import neighborhoodCouncil json
neighborhoodCouncil_json = geopandas.read_file('NC.geojson')

#spatial Joins neighborhoodCouncil to company
neighborhoodCouncil = neighborhoodCouncil_json.to_crs("EPSG:4326")
company = company.to_crs("EPSG:4326")
neighborhoodCouncil_frame = geopandas.GeoDataFrame(neighborhoodCouncil)
company_frame = geopandas.GeoDataFrame(company)
company_neighborhoodCouncil = geopandas.sjoin(company_frame, neighborhoodCouncil_frame, how="inner", op='intersects')

#count company in neighborhoodCouncil
count_company_neighborhoodCouncil = company_neighborhoodCouncil.groupby('name').size().reset_index(name='company_counts')

#change column value
v = count_company_neighborhoodCouncil['name']
del count_company_neighborhoodCouncil['name']
count_company_neighborhoodCouncil['NeighborhoodCouncil'] = v

#attribute Joins fire and company
count_fire_company = count_company_neighborhoodCouncil.merge(count_fire_neighborhoodCouncil, on='NeighborhoodCouncil')

#count fire rate
count_fire_company['fire_rate'] = count_fire_company.fire_counts/count_fire_company.company_counts

#choropleth map
import plotly.express as pe
map4 = pe.choropleth_mapbox( count_fire_company,
                             geojson=neighborhoodCouncil_json,
                             locations="NeighborhoodCouncil",
                             color="fire_rate",
                             featureidkey='properties.name',
                             mapbox_style="carto-positron",
                             zoom=10,
                             center={'lat':47.24, 'lon':-122.44})
map4.update_layout(title_text='Fire Per Business License by Neighborhood Council',)
map4

### Problem 5 Fire Insurance 2 - Where should I build my bar to avoid fires? (4pts)

This one is, on the surface, even sillier than before; but, underneath it are some interesting questions. In what neighborhood council area should I found a bar if my **sole** goal is to minimize my risk of fires as can be modeled out of the data you currently have?

*That's a pretty vague question*. Notice, I'm **not** asking you for a map here. Rather, I want you to conduct some analysis and then **explain that analysis**. Perhaps, you think I'd be happiest starting my bar where the fewest fires per bar have occurred in the past five years? Perhaps, you think I should start where there are a low number of total bars **and** a low level of property **lost** to fire (i.e. it doesn't matter how many fires there were, but how much damage they did)? Perhaps, you need to go get some other data to factor into your analysis.

**Here is what you need for this problem:**
1. **Documented** code that analyzes at least **two** factors for where I should establish my bar.
2. **A paragraph or two** that explains why the factors you've chosen are important and additionally describes the limitations you see in your approach.

You get 2 pts for each section. 
**Bonus:** Include additional data (+1pt per data source, +1pt per additional factor analyzed) - no limit, but also no bonus for obviously worthless data ('the most dog parks are in this area!')



In [ ]:
#factor 1

#import fire in the past five years(from 2016-2020)
fire = pandas.read_csv('Fires.csv')
fire_5years = fire[fire.IncidentYear >= 2016]

#count fire amount in each neighborhood council 
count_fire_neighborhoodCouncil = fire_5years.groupby('NeighborhoodCouncil').size().reset_index(name='fire_counts')

#import bar
x = 0
while x <33000:
    url = 'https://data.cityoftacoma.org/resource/29fz-3eiz.geojson?$limit=1000&$offset='+str(x)
    if x == 0:
        df = geopandas.read_file(url)
    else:    
        df2 = geopandas.read_file(url)
        df = df.append(df2)
    x = x + 1000
company = df[df['city'] == 'TACOMA']
bar = company[company['naics_code'] == '722410']

#import neighborhoodCouncil json
neighborhoodCouncil_json = geopandas.read_file('NC.geojson')

#spatial Joins neighborhoodCouncil to bar
neighborhoodCouncil = neighborhoodCouncil_json.to_crs("EPSG:4326")
bar = bar.to_crs("EPSG:4326")
neighborhoodCouncil_frame = geopandas.GeoDataFrame(neighborhoodCouncil)
bar_frame = geopandas.GeoDataFrame(bar)
company_neighborhoodCouncil = geopandas.sjoin(bar_frame, neighborhoodCouncil_frame, how="inner", op='intersects')

#count bar in neighborhoodCouncil
count_bar_neighborhoodCouncil = company_neighborhoodCouncil.groupby('name').size().reset_index(name='bar_counts')

#change column value
v = count_bar_neighborhoodCouncil['name']
del count_bar_neighborhoodCouncil['name']
count_bar_neighborhoodCouncil['NeighborhoodCouncil'] = v

#attribute Joins fire and bar
count_fire_bar = count_bar_neighborhoodCouncil.merge(count_fire_neighborhoodCouncil, on='NeighborhoodCouncil')

#count fire rate per year and sort
count_fire_bar['fire_rate_per_year'] = count_fire_bar.fire_counts/count_fire_bar.bar_counts/5
count_fire_bar = count_fire_bar.sort_values('fire_rate_per_year')
count_fire_bar.head(10)

**Factor 1 - Fire rate per bar per year:** North End is the lowest.

In [ ]:
#factor 2

#sum fire loss in each neighborhood council 
count_fireloss_neighborhoodCouncil = fire_5years.groupby('NeighborhoodCouncil')['EstimatedTotalFireLoss'].sum().reset_index(name='fire_lost_sum')

#attribute Joins fireloss and bar
count_fireloss_bar = count_fireloss_neighborhoodCouncil.merge(count_bar_neighborhoodCouncil, on='NeighborhoodCouncil')

#count fireloss per bar per year
count_fireloss_bar['fireloss_per_bar'] = count_fireloss_bar.fire_lost_sum/count_fireloss_bar.bar_counts/5

#attribute Joins fireloss and fire rate
count_fireloss_bar = count_fireloss_bar.merge(count_fire_bar, on='NeighborhoodCouncil')

#sort
count_fireloss_bar = count_fireloss_bar.sort_values('fireloss_per_bar')
count_fireloss_bar.head(10)

**Factor 2 - Fire loss rate per bar per year:** Central is the lowest.

In [ ]:
#factor 3

#change crs to Washington North EPSG:32148
neighborhoodCouncil = neighborhoodCouncil_json.to_crs("EPSG:32148")

#count neighborhood Council area per square kliometers
neighborhoodCouncil["area"] = neighborhoodCouncil['geometry'].area/1000000

#change column value
v = neighborhoodCouncil['name']
del neighborhoodCouncil['name']
neighborhoodCouncil['NeighborhoodCouncil'] = v

#attribute Joins bar and neighborhood council area
count_bar_area = count_bar_neighborhoodCouncil.merge(neighborhoodCouncil, on='NeighborhoodCouncil')

#count bar per square meters
count_bar_area['bar_per_square_kilometers'] = count_bar_area.bar_counts/count_bar_area.area
count_bar_area = count_bar_area.sort_values('bar_per_square_kilometers')
count_bar_area.head(10)

**Factor 3 - Bar per square kilometers:** West End is the lowest.

In [ ]:
#factor 4

#attribute Joins fire and neighborhood council area
count_fire_area = count_fire_neighborhoodCouncil.merge(neighborhoodCouncil, on='NeighborhoodCouncil')

#count fire per square meters per year
count_fire_area['fire_per_square_kilometers_per_year'] = count_fire_area.fire_counts/count_fire_area.area/5
count_fire_area = count_fire_area.sort_values('fire_per_square_kilometers_per_year')
count_fire_area.head(10)

**Factor 4 - Fire rate per square kilometers per year:** North East is the lowest.

Factor 1: **Fire rate per bar per year** provides the result that fire incidents per bar in each neighborhood council per year. The bar in North End has the least fire occurred rate. But this analysis does not consider the fire size, the neighborhood council with the least fire rate is not equal to they has the least loss in the fire.

Factor 2: **Fire loss rate per bar per year** provides the estimated property loss per bar in each neighborhood council per year. I joined the attribute of the factor 1 result to compare. The rank is a bit different from factor 1. Central has the least property loss in fire incidents but its rank of fire rate per bar per year is third. Because the fire data only records the total fire incident amounts, both the result of factor 1 and factor 2 could not represent the risk of fire in or influence the bar.

In factor 3,**Bar per square kilometers** provides the density of the bar in each neighborhood council. The smaller value means fewer bar per square kilometers. The West End has the smallest density. But this analysis does not consider the population, the neighborhood council with the smaller bar density might mean fewer people willing to go to the bar in this area or the population in this neighborhood council is smaller.

Factor 4: **Fire rate per square kilometers per year** provide the result of the fire occurred rate per square kilometers in each neighborhood council per year. North East is the place with the lowest fire rate. I think the analysis is relatively objective because it does not consider the bar amount. If I only want to find a place to minimize my risk of fires, North East will be a great choice.

#### Write your answer here. You can just type in text, but feel free to [mark it up](https://www.markdownguide.org/cheat-sheet).